In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests

In [8]:
def weather_forecast(city = '臺北市'):    
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu') # 允許在無GPU的環境下執行，可選
    options.add_argument('--window-size=1920x1080') # 建議設定
    browser = webdriver.Chrome(options=options)
    browser.get('https://www.cwb.gov.tw/V8/C/W/County/index.html')
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    i = 1
    try:
        while len(soup.select(f'#town > li:nth-child({i}) > a > span.city')) == 1:
            if city == soup.select(f'#town > li:nth-child({i}) > a > span.city')[0].text:
                data = soup.select('#town > li:nth-child(2) > a > span.city')[0].find_next_sibling('span')
                weather = data.find('img')['alt']
                tem = data.find('span', {'class':'tem-C is-active'}).text
            i += 1
        return city, weather, tem
    except:
        print('請輸入正確城市名稱')

In [13]:
line_token = 'gWLnjEjUdfzB86audQSBc7pu9t00EFGeciT0EBuBRM8'
def lineNotify(city, weather, tem, line_token):
    msg = f"{city}：{weather}，氣溫{tem}度"
    stickerPackageId = 2
    stickerId = 34
    url = "https://notify-api.line.me/api/notify"
    headers = {
        "Authorization": "Bearer " + line_token
    }
   
    payload = {"message": msg, "stickerPackageId": stickerPackageId, 'stickerId': stickerId}
    r = requests.post(url, headers = headers, params = payload)
    return r.status_code

In [14]:
lineNotify(*weather_forecast('高雄市'), line_token=line_token)

200